# Processar horas

In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
discrete_hours = None # valida se as horas serão discretas ou contínuas 
df_input_path = '../../dados/X_trainToronto.csv'

In [ ]:
def literal_eval(x):
    if pd.isna(x):
        return dict()
    raw_dict = ast.literal_eval(x)
    for key in raw_dict:
        raw_dict[key] = ast.literal_eval(raw_dict[key])
    return raw_dict
        
def proccess_df(df):
    df['hours'] = df['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else dict())

In [ ]:
df_reg = pd.read_csv(df_input_path)
proccess_df(df_reg)
df_reg = df_reg[['business_id', 'hours']]
df_reg

## Tratamento das horas abertas

In [40]:
from datetime import datetime, timedelta

midnight_tomorrow = datetime.strptime("0:0", "%H:%M") + timedelta(days=1)
midnight_today = datetime.strptime("0:0", "%H:%M")
daysOfWeek = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
times_of_day = {
         'morning': (datetime.strptime('6:0', "%H:%M"), datetime.strptime('12:0', "%H:%M")), 
         'afternoon': (datetime.strptime('12:0', "%H:%M"), datetime.strptime('18:0', "%H:%M")), 
         'evening': (datetime.strptime('18:0', "%H:%M"), midnight_tomorrow),
         'night': (midnight_today, datetime.strptime('6:0', "%H:%M"))
        }

def fill_hours(obj):
    result = {}
    for dayOfweek in daysOfWeek:
        hour1 = datetime.strptime("0:0", "%H:%M")
        hour2 = datetime.strptime("0:0", "%H:%M")
        if dayOfweek in obj:
            hours_obj = obj[dayOfweek].split('-')
            hour1 = datetime.strptime(hours_obj[0], "%H:%M")
            hour2 = datetime.strptime(hours_obj[1], "%H:%M")
            if hour1 >= hour2:
                hour2 += timedelta(days=1)
        for key in times_of_day:
            d1 = times_of_day[key][0]
            d2 = times_of_day[key][1]
            
            value = calculate_hoursOfDay(hour1, hour2, d1, d2)
            result[f'business_open_{dayOfweek}_{key}'] = value
    return result
            
    
def calculate_hoursOfDay(hour1: datetime, hour2: datetime, d1: datetime, d2: datetime):
    if hour1 > d2 or hour2 < d1:
        if hour2.day == midnight_tomorrow.day and (hour1 != midnight_tomorrow and hour2 != midnight_tomorrow):
            day1 = calculate_hoursOfDay(midnight_tomorrow, hour2, d1 + timedelta(days=1), d2 + timedelta(days=1))
            day2 = calculate_hoursOfDay(hour1, midnight_tomorrow, d1, d2)
            return day1 + day2
        return 0.0
        
    inf = hour1
    sup = hour2
    if hour1 <= d1:
        inf = d1
    if hour2 >= d2:
        sup = d2
        
    if discrete_hours == True and float((sup - inf).seconds) > 0.001:
        return 1.0
    return float((sup - inf).seconds) / 21600.0

In [41]:
hours_features = pd.DataFrame(list(df_reg['hours'].apply(fill_hours)))
df_reg = pd.concat([df_reg, hours_features], axis=1)
df_reg

,business_id,latitude,longitude,attributes,categories,hours,destaque,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,...,business_open_Saturday_evening,business_open_Saturday_night,business_open_Sunday_morning,business_open_Sunday_afternoon,business_open_Sunday_evening,business_open_Sunday_night,business_open_Wednesday_morning,business_open_Wednesday_afternoon,business_open_Wednesday_evening,business_open_Wednesday_night
0,vHzWmPWHN4J1hRR3W3AMQg,43.648977,-79.420495,"{'Ambience': {'romantic': False, 'intimate': F...","[wine bars, tapas/small plates, restaurants, b...","{'Monday': '18:0-23:0', 'Tuesday': '18:0-23:0'...",0,True,False,False,...,0.750000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.833333,0.000000
1,15to24Q-otAHmto7FzsWRg,43.654002,-79.452189,"{'BusinessParking': {'garage': False, 'street'...","[beauty & spas, hair salons, barbers, shopping...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.500000,0.833333,0.000000,0.000000
2,8aqKdf4G4AAir8k_Kdslvg,43.648493,-79.383214,"{'ByAppointmentOnly': True, 'AcceptsInsurance'...","[health & medical, medical centers]","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.833333,1.000000,0.333333,0.000000
3,uxU1vr5AhhkTQ83X0bpeyg,43.772453,-79.448136,{'ByAppointmentOnly': False},"[health & medical, hospitals, medical centers]",{},0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,f702hTJoqdR34Jn23C7d1A,43.665082,-79.460800,{'RestaurantsPriceRange2': 3},"[automotive, bikes, shopping, auto repair, spo...",{},0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,43.776237,-79.235318,"{'Caters': True, 'BikeParking': False, 'WiFi':...","[restaurants, halal, mexican]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",0,False,True,True,...,0.833333,0.000000,0.000000,1.0,0.833333,0.0,0.166667,1.000000,0.833333,0.000000
17578,o2811xZ0ufRi7WcsqWiGzw,43.670098,-79.388198,"{'RestaurantsPriceRange2': 2, 'BikeParking': F...","[food, coffee & tea]",{},0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
17579,v9T9m_TbpLuaC2Kud5UbLA,43.679862,-79.340900,"{'RestaurantsPriceRange2': 2, 'BikeParking': T...","[sewing & alterations, shopping, local services]","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'...",0,False,False,False,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.333333,1.000000,0.000000,0.000000
17580,bzGnAIe9OiBHGWJE-dZ_8g,43.745578,-79.220423,"{'BusinessParking': {'garage': False, 'street'...","[chicken wings, bars, nightlife, restaurants, ...","{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'...",0,False,True,False,...,1.000000,0.333333,0.083333,1.0,1.000000,0.0,0.083333,1.000000,1.000000,0.166667


In [ ]:
df_reg.to_parquet(product['data'])

## Teste manual

In [ ]:
hours_columns = [x for x in df_reg.columns if x.startswith('business_open_')]
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['hours'])
    display(df_reg[hours_columns].iloc[[13]])
    display('----------------------------------')
    display(df_reg.iloc[542]['hours'])
    display(df_reg[hours_columns].iloc[[542]])
    display('----------------------------------')
    display(df_reg.iloc[5389]['hours'])
    display(df_reg[hours_columns].iloc[[5389]])